In [53]:
import camelot

In [54]:
import pymongo
con = pymongo.MongoClient()
coll = con.alstom.data

In [57]:
pages = [str(i) for i in range(23,53,2)]

In [58]:
tables = camelot.read_pdf('Images/demo.pdf', pages=','.join(pages))

In [66]:
try:
    mongo_id = int(coll.find_one(sort=[("_id", -1)])["_id"])
except:
    mongo_id = 0  

In [67]:
i = 0
for table in tables:
    indexs = table.df[0][0].lower().split("\n")
    #print(indexs)
    
    title = indexs[1].strip()
    tabella = indexs[2].strip().split("tavola ")[1].replace("/","")
    edition = indexs[4].strip().split("edizione ")[1]
    page = indexs[5].strip().split("pag.  ")[1]
    
    descriptions = table.df[1][4].split(" \n")
    ids = table.df[2][4].split(" \n")
    suppliers = table.df[3][4].split(" \n")
    documents = table.df[4][4].split(" \n")
    indentifiers = table.df[5][4].split(" \n")
    pieces = table.df[7][4].split(" \n")
    notes = table.df[9][4].split(" \n")
    
    
    if len(descriptions) == len(ids):
        for id, value in enumerate(ids, start=0):
            mongo_id += 1            
            json = {"_id" : mongo_id, 
                        "macro_id" : tabella, 
                        "table_title" : title,
                        "table_edition" : edition,
                        "table_page" : page,
                        "pdf_table_pages" : int(pages[i]), 
                        "pdf_image_pages" : int(pages[i])+1,
                        
                        "id" : value, 
                        "description" : descriptions[id],
                        "supplier" : suppliers[id],
                        "document" : documents[id],
                        "indentifier" : indentifiers[id],
                        "piece" : pieces[id]
                       }
            try:
                #print(json)
                coll.insert_one(json)
            except:
                print(json, "duplicate key")
            
    i += 1